Run the RCF filter on some set of recent alerts, match with some catalogs and run SNguess and Parsnip. Hopefully submit to Fritz.

In [ ]:
import requests, os
from astropy.time import Time

In [ ]:
token = os.environ["ARCHIVE_TOKEN"]
header = {"Authorization": "bearer "+token}
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.RcfFilter import RcfFilter

from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
logger = AmpelLogger.get_logger()

## A. Define filter parameters
Default RCF 2020B values.

In [ ]:
# Decent filter parameters
filter_config = {
    "min_ndet": 1,          # Min number of detections
    "max_magpsf": 19.,      
    "min_dist_to_sso": 15,     
    "min_gal_lat": 7,      
    "min_age": 0.02,
    "max_ipac_age": 0,
    "min_rb": 0,
    "min_drb": 0,
}

In [ ]:
t0filter = RcfFilter( **filter_config, logger=logger )
t0filter.post_init()

## B. Getting a handle to alerts.
A _resume token_, which points to a specific set of alerts. We here present three ways of doing this. 

In [ ]:
delta_t = 1.

In [ ]:
# Adding some handles to make it feasible...
query = {
  "jd": {
    "$gt": Time.now().jd-delta_t,
    "$lt": Time.now().jd
  },
  "candidate": {
    "rb": {
      "$gt": 0.2
    },
    "drb": {
      "$gt": 0.95   
    },
    "magpsf": {
      "$lt": 20,
      "$gt": 18
    },
    "ra": {
      "$lt": 51.7,
      "$gt": 51.6
    },
    "dec": {
      "$lt": 9.19,
      "$gt": 9.18
    },
    "ndethist": {
      "$gt": 0,
      "$lte": 7
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
# Connection
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'
#endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?'
header = {"Authorization": "bearer "+os.environ["ARCHIVE_TOKEN"]}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
response.ok

In [ ]:
resume_token = response.json()['resume_token']

In [ ]:
resume_token = 'uip_MPLU-XYNH_C9EU_loVdIdtnEFbDU6XVUwlYK1ms'

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel83/Ampel-HU-astro/ampel_conf.yaml'

In [ ]:
channel = 'RCFnb'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'dropme',
    purge_db = True,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
# Will use NED for spectroscopic redshifts.
cat_conf = {
    'catalogs' : {
        'SDSS_spec' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['z', 'bptclass', 'subclass'],
            'all': False,
        },
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'GLADEv23' : {
            'use' : 'extcats',
            'rs_arcsec' : 10,
            'keys_to_append' : ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },
         'PS1_photoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['raMean','decMean','z_phot','z_photErr', 'z_phot0', '_2'],
            'pre_filter' : None,
            'post_filter' : None,
        },
    }
}

In [ ]:
# New, new, also with all categories?
ampelz_conf = {
    "max_redshift_category" : 7,
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": cat_conf,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },
#        {
#            "unit": "T2LSPhotoZTap",
#            "config": dl_config,
#            "link_override": {
#                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
#                }
#        },

    ]
}

In [ ]:
sncosmo_conf = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
#    "unc" : 3,
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
    'tabulator': [
        {'unit': 'ZTFT2Tabulator'},
    ],
    "plot_props": {
        "tags": ["SALT", "SNCOSMO"],
        "file_name": {
            "format_str": "%s_%s_%s.svg",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "title": {
            "format_str": "%s %s %s",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "fig_text": {
            "format_str": "%s %s \nz-source %s \nchisq %.2f ndof %s",
            "arg_keys": ["stock", "model", "redshift_kind", "chisq", "ndof"]
        },
        "width": 10,
        "height": 6,
#        "id_mapper": "ZTFIdMapper",
        "id_mapper": None,
        "disk_save": "/home/jnordin/tmp/ztfsalt",
    }
}

In [ ]:
parsnip_config = {
    'parsnip_model':'/home/jnordin/data/parsnip/parsnip_ps1_ztf.pt', 
    'parsnip_classifier':'/home/jnordin/data/parsnip/ztf_classifier.pkl', 
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
    'plot_suffix': 'png',
    'plot_dir': '/home/jnordin/tmp/ztfparsnip',
    'tabulator': [
        {'unit': 'ZTFT2Tabulator'},
    ]
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "RcfFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2DigestRedshifts",
                                    "config": ampelz_conf,
                                },
                                {
                                    "unit": "T2MatchBTS",
                                },
                                {
                                    "unit": "T2BrightSNProb",
                                },
                                {
                                    "unit": "T2RunSncosmo",
                                    "config": sncosmo_conf,
                                },                            
                                {
                                    "unit": "T2RunParsnip",
                                    "config": parsnip_config,
                                },                            
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
#                            {
#                                'unit': 'T2LSPhotoZTap',
#                                'config': dl_config,
#                                "ingest": {
#                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
#                                }
#                            },
                        ],
                    }

                }
            }
        }
    ]

In [ ]:
response.json()

In [ ]:
loader_config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":response.json()['resume_token']}

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = 1000000000,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': None,
            'loader': {
                'unit': 'ZTFArchiveAlertLoader',
                'config': loader_config
            }
        }
    },
    directives = directives
)


In [ ]:
n = ac.run()

In [ ]:
n

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

In [ ]:
table_config =  {
                                        'table_schema' : {
                                            'T2DigestRedshifts': {
                                                'Ampel z' : ['ampel_z'],
                                                'Ampel z group' : ['group_z_nbr'],
                                                'Ampel distance' :['ampel_dist'],
                                            },
                                            'T2BrightSNProb' : {
                                                'RiseDecline_Success' : ['success'],
                                                'RiseDecline_SNguess' : ['SNGuess'],
                                                'RiseDecline_SNBool' : ['SNGuessBool'],
                                                'RiseDecline_nDet' : ['ndet'],
                                            },
                                            'T2MatchBTS' : {
                                                'Type' : ['bts_type'],
                                                'IAU ID': ['bts_IAUID'],
                                            },
                                            'T2RunSncosmo' : {
                                                'Peak B abs mag' : ['fit_metrics','restpeak_model_absmag_B'],
                                                'SALT2 X1' : ['sncosmo_result','paramdict','x1'],
                                                'SALT2 X1 (err)' : ['sncosmo_result','errors','x1'],
                                                'SALT2 Color' : ['sncosmo_result','paramdict','c'],
                                                'SALT2 Color (err)' : ['sncosmo_result','errors','c'],
                                                'Pulls around peak' : ['fit_metrics','absmean_peak_pull'],
                                                'Det. around peak' : ['fit_metrics','nbr_peak_pulls'],
                                            },
                                            'T2RunParsnip' : {
                                                'Parsnip chi2dpf' : ['prediction','chi2pdf'],
                                                'Parsnip(SLSN)' : ['classification','SLSN'],
                                                'Parsnip(SNII)' : ['classification','SNII'],
                                                'Parsnip(SNIa)' : ['classification','SNIa'],
                                                'Parsnip(SNIbc)' : ['classification','SNIbc'],
                                                'Parsnip(TDE)' : ['classification','TDE'],
                                            },
                                            
                                        },
                                        'transient_table_schema' : {
                                        },
                                        'include_stock' : True,
                                        'include_channels' : True,
                                        'fmt' : 'csv',
                                        'local_path' : "/home/jnordin/tmp"
                                    }

In [ ]:
t3p = ctx.new_context_unit(
    process_name = "LensPrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3StockSelector",
                            "config": {"channel": channel}
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T1", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {
                                "unit": "TransientTablePublisher",
                                "config": table_config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()